## Determine new releases
Use requests and beautifulsoup4 to check and see which page on Ronin Flix site has the desired title, "Opera"


In [1]:
import requests
import bs4

In [2]:
NR_URL = 'https://roninflix.com/collections/new-release'
CS_URL = 'https://roninflix.com/collections/coming-soon'


In [3]:
#this first method should be in a separate script so you don't pull too much
def pull_site(url):
    raw_site_page = requests.get(url, timeout=5)
    raw_site_page.raise_for_status()
    return raw_site_page

def scrape(site):
    titles_list = []
    
    soup = bs4.BeautifulSoup(site.text, 'html.parser')
    select_list = soup.select('.grid-view-item__title')
    
    for titles in select_list:
        titles_list.append(titles.getText())
        
    return titles_list

In [9]:
def check_movie():
    new_titles = scrape(pull_site(NR_URL))
    coming_titles = scrape(pull_site(CS_URL))

    for title in new_titles:
        if 'Opera' in title:
            return 1
    for title in coming_titles:
        if 'Opera' in title:
            return 0
    return -1

def whereis_movie():
    movie_status = check_movie()
    
    if movie_status == 0:
        return(f'{title} is still on Ronin Coming Soon page')
    elif movie_status == 1:
        return(f'{title} found on Ronin New Releases page')
    else:
        return(f'Something is going on... You\'d better check Ronin to see if Opera has moved from Coming Soon.')
    

## email notification
Send summary via email

In [10]:
def email_notify(msg_body):
    import os
    import smtplib

    mail_acct = os.environ.get('ALERTS_EMAIL')
    mail_pass = os.environ.get('ALERTS_PASSWORD')

    with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
        #identify yourself
        smtp.ehlo()
        #encrypt connection
        smtp.starttls()
        smtp.ehlo()

        smtp.login(mail_acct, mail_pass)

        subject = 'Movie Notification'
        msg = f'Subject: {subject}\n\n{msg_body}'

        smtp.sendmail(mail_acct, 'grantryan.king@gmail.com', msg)

In [11]:
email_notify(whereis_movie())

## save activity
to stop process when complete